In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers==4.52.4

In [ ]:
from huggingface_hub import login
login()

In [4]:
pip install protobuf==3.20.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [8]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cuda:0


In [9]:
pip install youtube-transcript-api

Note: you may need to restart the kernel to use updated packages.


In [10]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Raises ValueError if no 'v' parameter is found.
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')

    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

if __name__ == "__main__":
    url = "https://www.youtube.com/watch?v=q1xNuU7gaAQ"  # Example valid URL

    video_id = extract_video_id(url)

    api = YouTubeTranscriptApi()
    fetched = api.fetch(video_id, languages=['en'])  # returns a FetchedTranscript

    text = "\n".join(snippet.text for snippet in fetched)
    print(text)


[Music]
hello friends
you might be wondering what i was doing
in that hole well that is one of my
usual trips to find something new
and guess what i found today
this is the bag of five senses
come let's see what's inside zoom in
whoa
these are the five senses
sense of sight sense of hearing sense of
smell sense of taste and sense of touch
these are the eyes
they help you see
yes you're using them right now while
looking at me
amazing isn't it
your eyes help you to see things
just like how it will help us to see
what's next
these other years
they help you hear
[Music]
do you hear that
wonderful music isn't it
your ears help you hear
so whenever you hear beautiful things
like birds chirping
or a car blowing its horn
you know it's your sense of hearing
[Music]
i smell something
is it coming from the garbage bin
outside
this is your nose
it helps you smell different things
it could be something stinky like the
garbage
or something delicious
like a mug of hot chocolate
[Music]
this is your 

In [11]:
pip install pyngrok

Note: you may need to restart the kernel to use updated packages.


In [12]:
NGROK_TOKEN = "35f7eKk4dud8PxL7IPbSrLXpOQB_4aEX5ygh5ZF5r7jtxZvcd"
API_KEY = "secret123"

In [40]:
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN #put token of ngrok
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://apprehensibly-cresylic-layla.ngrok-free.dev


INFO:     Started server process [187]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:57777 (Press CTRL+C to quit)


In [39]:
from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

API_KEY = "secret123"  # Replace with your actual key
app = FastAPI()

# -----------------------------
# Utility Functions
# -----------------------------
def extract_video_id(url: str) -> str:
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')
    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

def fetch_transcript(video_id: str) -> str:
    api = YouTubeTranscriptApi()
    fetched = api.fetch(video_id, languages=['en'])  # returns a FetchedTranscript
    return "\n".join(snippet.text for snippet in fetched)

def summarize_text(text: str, max_words: int = 200) -> str:
    words = text.split()
    return " ".join(words[:max_words]) + "..."

# -----------------------------
# Endpoint
# -----------------------------
@app.post("/summarize")
async def summarize(req: Request):
    # Authorization check
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    # Parse JSON payload
    data = await req.json()
    url = data.get("url")
    if not url:
        raise HTTPException(status_code=400, detail="Missing 'url' in request body")

    try:
        # Extract video ID and fetch transcript
        video_id = extract_video_id(url)
        transcript_text = fetch_transcript(video_id)

        # Summarize transcript
        summary = summarize_text(transcript_text)

        return {
            "video_id": video_id,
            "summary": summary
        }

    except ValueError as ve:
        raise HTTPException(status_code=400, detail=str(ve))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Transcript fetch failed: {str(e)}")

In [42]:
import requests

API_KEY = "secret123"
url = "https://apprehensibly-cresylic-layla.ngrok-free.dev/summarize"

payload = {"url": "https://www.youtube.com/watch?v=q1xNuU7gaAQ"}
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.post(url, json=payload, headers=headers, verify=False)
if response.status_code == 200:
    print("Summary:", response.json()["summary"])
else:
    print("Error:", response.status_code, response.text)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apprehensibly-cresylic-layla.ngrok-free.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


INFO:     34.80.12.162:0 - "POST /summarize HTTP/1.1" 200 OK
Summary: [Music] hello friends you might be wondering what i was doing in that hole well that is one of my usual trips to find something new and guess what i found today this is the bag of five senses come let's see what's inside zoom in whoa these are the five senses sense of sight sense of hearing sense of smell sense of taste and sense of touch these are the eyes they help you see yes you're using them right now while looking at me amazing isn't it your eyes help you to see things just like how it will help us to see what's next these other years they help you hear [Music] do you hear that wonderful music isn't it your ears help you hear so whenever you hear beautiful things like birds chirping or a car blowing its horn you know it's your sense of hearing [Music] i smell something is it coming from the garbage bin outside this is your nose it helps you smell different things it could be something stinky like the garbage or